# Running Monte Carlo Transport Independently

This tutorial demonstrates how to run the Monte Carlo transport loop directly using `Simulation.from_config` without running full TARDIS iterations. This approach gives you direct control over the Monte Carlo transport process.

In [2]:
from pathlib import Path

import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.io.configuration.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import (
    initialize_estimator_statistics,
)
from tardis.transport.montecarlo.montecarlo_main_loop import (
    montecarlo_main_loop,
)
from tardis.transport.montecarlo.packets.trackers import (
    generate_rpacket_last_interaction_tracker_list,
    rpacket_last_interaction_tracker_list_to_dataframe,
)

In [3]:
# User-configurable variables
CONFIG_FILE_NAME = "tardis_example.yml"
NUMBER_OF_PACKETS = 10000
NUMBER_OF_VPACKETS = 0  # Set to 0 to disable virtual packets
ITERATION_NUMBER = 1
SHOW_PROGRESS_BARS = True
TOTAL_ITERATIONS = 1
ENABLE_RPACKET_TRACKING = False  # True: full tracking, False: last interaction only

In [4]:
# Setup simulation state from config
config_file = Path(CONFIG_FILE_NAME)
if not config_file.exists():
    raise FileNotFoundError(f"Configuration file {CONFIG_FILE_NAME} not found")

config = Configuration.from_yaml(str(config_file))
atom_data = AtomData.from_hdf("kurucz_cd23_chianti_H_He_latest.h5")
sim = Simulation.from_config(config, atom_data=atom_data)

print("Simulation created successfully!")

WARNING	Task(Task-3) tardis.io.model.parse_composition_configuration:parse_composition_configuration.py:parse_density_from_config()- Number of density points larger than number of shells. Assuming inner point irrelevant
WARNING	Task(Task-3) tardis.io.model.parse_mass_fraction_configuration:parse_mass_fraction_configuration.py:parse_mass_fractions_from_config()- model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Simulation created successfully!


In [5]:
# Initialize opacity and macro atom states manually
sim.opacity_state = sim.opacity.legacy_solve(sim.plasma)

if sim.macro_atom is not None:
    sim.macro_atom_state = sim.macro_atom.solve(
        sim.plasma.j_blues,
        sim.plasma.atomic_data,
        sim.opacity_state.tau_sobolev,
        sim.plasma.stimulated_emission_factor,
        sim.opacity_state.beta_sobolev,
    )
else:
    sim.macro_atom_state = None

print("Opacity and macro atom states initialized!")

Opacity and macro atom states initialized!


In [6]:
# Extract states from simulation
geometry_state = sim.simulation_state.geometry
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Initialize estimators
tau_sobolev_shape = opacity_state.tau_sobolev.shape
gamma_shape = (0, geometry_state.no_of_shells)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Convert to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(sim.macro_atom_state, line_interaction_type)

print("Monte Carlo states prepared!")

Monte Carlo states prepared!


In [7]:
# Create packet collection
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

# Setup packet tracking
if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_rpacket_tracker_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
    rpacket_tracker_collection = None
    print("Using full RPacket tracking")
else:
    # Initialize the last interaction tracker collection
    # Generate individual trackers for the main loop
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )
    print("Using last interaction tracking with collection")

Using last interaction tracking with collection


In [8]:
# Run the Monte Carlo main loop
v_packets_energy_hist, last_interaction_tracker, vpacket_tracker = (
    montecarlo_main_loop(
        packet_collection,
        geometry_state_numba,
        time_explosion,
        opacity_state_numba,
        montecarlo_configuration,
        estimators,
        spectrum_frequency_grid,
        rpacket_trackers,
        NUMBER_OF_VPACKETS,
        SHOW_PROGRESS_BARS,
    )
)


/Users/wkerzend/python/tardis/tardis/transport/montecarlo/montecarlo_main_loop.py:150: NumbaTypeSafetyWarning:

unsafe cast from uint64 to int64. Precision may be lost.



In [9]:
tracker = rpacket_trackers[0]

In [11]:
tracker_df = rpacket_last_interaction_tracker_list_to_dataframe(rpacket_trackers)

In [12]:
tracker_df

,last_interaction_type,last_line_interaction_in_id,last_line_interaction_in_nu,last_line_interaction_out_id
0,CONTINUUM_PROCESS,-1,5.150244e+14,-1
1,CONTINUUM_PROCESS,-1,1.031473e+15,-1
2,CONTINUUM_PROCESS,-1,1.188678e+15,-1
3,CONTINUUM_PROCESS,-1,7.361418e+14,-1
4,NO_INTERACTION,-1,2.763790e+14,-1
...,...,...,...,...
9995,NO_INTERACTION,-1,4.272588e+14,-1
9996,NO_INTERACTION,-1,1.888737e+14,-1
9997,NO_INTERACTION,-1,3.801189e+14,-1
9998,NO_INTERACTION,-1,7.187764e+14,-1


In [ ]:
# Convert tracker data to DataFrame for analysis
if ENABLE_RPACKET_TRACKING:
    # For full tracking, use the built-in conversion from RPacketCollection
    if hasattr(last_interaction_tracker, 'to_dataframe'):
        tracker_df = last_interaction_tracker.to_dataframe()
        print(f"Full tracking DataFrame shape: {tracker_df.shape}")
        print("Full tracking DataFrame columns:", list(tracker_df.columns)[:10], "...")
    else:
        print("Full tracking data available but no DataFrame conversion method")
else:
    # For last interaction tracking, convert the tracker list to DataFrame
    tracker_df = rpacket_last_interaction_tracker_list_to_dataframe(rpacket_trackers)
    print(f"Last interaction tracking DataFrame shape: {tracker_df.shape}")
    print("Last interaction tracking DataFrame columns:", list(tracker_df.columns))

    # Display some sample data
    print("\nSample interaction data:")
    print(tracker_df.head())

    # Show interaction type distribution (now categorical and human-readable)
    interaction_counts = tracker_df['last_interaction_type'].value_counts()
    print("\nInteraction type distribution:")
    for interaction_type, count in interaction_counts.items():
        print(f"  {interaction_type}: {count} packets")

    # Show line interaction statistics
    line_interactions = tracker_df[tracker_df['last_interaction_type'] == 'LINE']
    if len(line_interactions) > 0:
        print("\nLine interaction details:")
        print(f"  Total line interactions: {len(line_interactions)}")
        print(f"  Unique line IDs (input): {line_interactions['last_line_interaction_in_id'].nunique()}")
        print(f"  Unique line IDs (output): {line_interactions['last_line_interaction_out_id'].nunique()}")

    print("\n✓ DataFrame with categorical interaction types ready for analysis!")

In [11]:
test.line_before_id

-1

In [ ]:
# Finalize the tracker collection and access results
if not ENABLE_RPACKET_TRACKING and rpacket_tracker_collection is not None:
    # Finalize the collection to consolidate data into arrays
    rpacket_tracker_collection.finalize(rpacket_trackers)

    print("Tracker Collection Results:")
    print(f"Number of packets tracked: {rpacket_tracker_collection.num_packets}")
    print(f"Collection is finalized: {rpacket_tracker_collection.is_finalized}")

    # Access some example data from the collection
    print(f"Final radii range: {rpacket_tracker_collection.final_r.min():.2e} - {rpacket_tracker_collection.final_r.max():.2e} cm")
    print(f"Line interactions total: {rpacket_tracker_collection.line_interactions_count.sum()}")
    print(f"Electron scattering total: {rpacket_tracker_collection.escat_interactions_count.sum()}")

    # Convert to DataFrame for analysis
    tracker_df = rpacket_tracker_collection.to_dataframe()
    print(f"DataFrame shape: {tracker_df.shape}")
    print("DataFrame columns:", list(tracker_df.columns))

In [15]:
test = rpacket_trackers[0]

0.0

In [8]:
# Inspect results
print("Monte Carlo Results:")
print(f"V-packet energy histogram shape: {v_packets_energy_hist.shape}")
print(f"Last interaction tracker type: {type(last_interaction_tracker)}")
if ENABLE_RPACKET_TRACKING:
    print(f"V-packet tracker available: {type(vpacket_tracker)}")

# Results are now available in the notebook for further analysis:
# - v_packets_energy_hist: energy histogram of virtual packets
# - last_interaction_tracker: final interaction data for all packets
# - vpacket_tracker: virtual packet collection (if enabled)
# - sim: original simulation object

Monte Carlo Results:
V-packet energy histogram shape: (10001,)
Last interaction tracker type: <class 'numba.experimental.jitclass.boxing.LastInteractionTracker'>


## Usage Instructions

### Configuration Options:
- **NUMBER_OF_PACKETS**: Number of packets to simulate (default: 10000)
- **NUMBER_OF_VPACKETS**: Number of virtual packets per interaction (0 = disabled)
- **ENABLE_RPACKET_TRACKING**: True for full tracking, False for last interaction only

### For Development and Debugging:
1. Set `NUMBA_DISABLE_JIT=1` in your environment variables to disable Numba JIT compilation
2. Set breakpoints in cell 8 (the Monte Carlo main loop call) 
3. Run the notebook in debug mode to step through the transport process

### Key Advantages Over Full TARDIS Run:
- **Direct control**: Access Monte Carlo transport without full simulation iterations
- **Manual state initialization**: All states explicitly prepared for transparency
- **Clean separation**: Setup and execution are clearly separated
- **Educational**: Perfect for understanding Monte Carlo transport physics step-by-step

# Running Monte Carlo Transport Loop

This tutorial demonstrates how to run the `montecarlo_main_loop` function directly with the TARDIS quickstart configuration.

In [ ]:
from pathlib import Path

import astropy.units as u

from tardis import run_tardis
from tardis.transport.montecarlo.montecarlo_main_loop import (
    montecarlo_main_loop,
)
from tardis.transport.montecarlo.packets.packet_trackers import (
    generate_rpacket_last_interaction_tracker_list,
    generate_rpacket_tracker_list,
)

/Users/wkerzend/python/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [ ]:
print("Monte Carlo Results:")
print(f"V-packet energy histogram shape: {v_packets_energy_hist.shape}")
print(f"Last interaction tracker type: {type(last_interaction_tracker)}")
if ENABLE_RPACKET_TRACKING:
    print(f"V-packet tracker available: {type(vpacket_tracker)}")

In [3]:
sim = run_tardis(CONFIG_PATH, show_progress_bars=SHOW_PROGRESS_BARS)

Auto-detected VSCode environment


Auto-detected VSCode environment


Initializing tabulator and plotly panel extensions for widgets to work


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:\u2007\u2007\u2007', layout=Layout(width='6%')), FloatProgress(value=0.…

Embedding the final state for Jupyter environments


In [ ]:
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import (
    initialize_estimator_statistics,
)

# Get the geometry from simulation_state
geometry_state = sim.simulation_state.geometry
# Get the opacity state from the simulation (not from transport.transport_state)
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value

# Initialize estimators using the same function used in TARDIS
tau_sobolev_shape = opacity_state.tau_sobolev.shape
# For continuum estimators, check if we have continuum processes
if hasattr(opacity_state, 'continuum_state') and opacity_state.continuum_state is not None:
    gamma_shape = (opacity_state.continuum_state.photo_ion_idx.max() + 1, geometry_state.no_of_shells)
else:
    gamma_shape = (0, geometry_state.no_of_shells)

estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Convert objects to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
macro_atom_state = sim.macro_atom_state if hasattr(sim, 'macro_atom_state') else None
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(macro_atom_state, line_interaction_type)

In [ ]:
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_rpacket_tracker_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
else:
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )

Using full RPacket tracking - detailed packet trajectories


In [26]:
%%timeit
# Run the Monte Carlo main loop
# Reinitialize estimators (they get modified by the main loop)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Run the Monte Carlo main loop
v_packets_energy_hist, last_interaction_tracker, vpacket_tracker = montecarlo_main_loop(
    packet_collection,
    geometry_state_numba,
    time_explosion,
    opacity_state_numba,
    montecarlo_configuration,
    estimators,
    spectrum_frequency_grid,
    rpacket_trackers,
    NUMBER_OF_VPACKETS,
    ITERATION_NUMBER,
    SHOW_PROGRESS_BARS,
    TOTAL_ITERATIONS,
)



242 ms ± 4.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Tracking Options Explained (TARDIS Implementation)

### Important: LastInteractionTracker is Always Available
The `montecarlo_main_loop` **always** returns a `LastInteractionTracker` object containing the final interaction data for all packets, regardless of the tracking option chosen.

### Option 1: Full RPacket Tracking (`ENABLE_RPACKET_TRACKING = True`)
- **Equivalent to**: `config.montecarlo.tracking.track_rpacket = True` in TARDIS config
- **Function used**: `generate_rpacket_tracker_list()`
- **RPacket Data**: Records position (r), frequency (nu), direction (mu), energy, shell_id, and interaction_type at **each step** of every packet's journey
- **LastInteractionTracker**: Also populated with final interaction data
- **Use case**: Detailed analysis of packet trajectories, debugging transport physics, studying packet paths
- **Memory**: Higher memory usage as it stores the complete history for each packet

### Option 2: Last Interaction Collection (`ENABLE_RPACKET_TRACKING = False`)  
- **Equivalent to**: `config.montecarlo.tracking.track_rpacket = False` in TARDIS config (default)
- **Implementation**: Uses `RPacketLastInteractionTrackerCollection` with individual `RPacketLastInteractionTracker` objects
- **RPacket Data**: Each packet gets a tracker that records detailed interaction data (before/after states for each interaction type)
- **Collection Features**: 
  - Automatic finalization into structured arrays
  - DataFrame conversion for analysis
  - Interaction statistics and summaries
- **LastInteractionTracker**: Contains aggregated final interaction data for all packets
- **Use case**: Statistical analysis of interactions, line formation regions, interaction type studies
- **Memory**: Moderate memory usage - more than basic tracking but less than full trajectory tracking

### New Collection Architecture Benefits:
- **Structured data access**: All tracker data organized in collection with methods
- **Analysis ready**: Built-in DataFrame conversion and statistics
- **Memory efficient**: Optimized field storage (removes redundant data)
- **Extensible**: Easy to add new interaction tracking features

**To switch between options:** Change `ENABLE_RPACKET_TRACKING` in cell 3 and re-run cells 7-9.

## Refactoring Recommendations for Monte Carlo Subsystem

Based on this tutorial, here are key areas that could benefit from refactoring for greater clarity:

### 1. **Inconsistent Object Access Patterns**
- **Current**: Mixed access through `sim.transport.transport_state`, `sim.opacity_state`, `sim.simulation_state`
- **Recommendation**: Create a unified interface or facade pattern that provides consistent access to all MC components

### 2. **Complex State Conversion**
- **Current**: Manual conversion to numba-compatible objects with required parameters
- **Recommendation**: Create helper functions or factory methods that handle numba conversions automatically

### 3. **Estimator Initialization Complexity** 
- **Current**: Manual shape calculation and estimator initialization
- **Recommendation**: Create an `EstimatorFactory` that handles all the shape calculations and initialization logic

### 4. **Packet Creation Indirection**
- **Current**: Multiple steps to create packets (source → create_packets → collection)
- **Recommendation**: Simplified packet creation API that reduces the number of steps

### 5. **Configuration Parameter Inconsistency**
- **Current**: Some parameters exist in config, others have defaults, naming is inconsistent
- **Recommendation**: Standardize configuration parameter naming and provide clear defaults

### 6. **Type System Clarity**
- **Current**: Mix of Python objects, Numba objects, with unclear conversion points
- **Recommendation**: Clear separation and documentation of when objects need to be converted

### 7. **Function Parameter Count**
- **Current**: `montecarlo_main_loop` takes 12+ parameters
- **Recommendation**: Create structured parameter objects or context managers

### Proposed Refactored Interface

Instead of the current complex setup, a cleaner interface could look like:

```python
# Proposed simplified API
from tardis.transport.montecarlo import MonteCarloRunner

# Create runner with simulation
mc_runner = MonteCarloRunner.from_simulation(sim)

# Configure run parameters
mc_config = MonteCarloRunConfig(
    number_of_packets=1000,
    number_of_vpackets=10,
    iteration=1,
    show_progress_bars=True
)

# Run the main loop with simplified interface
results = mc_runner.run_main_loop(mc_config)

# Access results with clear structure
print(f"V-packets histogram: {results.vpacket_energy_histogram.shape}")
print(f"Interaction tracker: {results.last_interaction_tracker.count}")
```

This would encapsulate all the complexity of:
- State extraction and conversion
- Estimator initialization  
- Packet creation and tracking
- Numba object preparation

In [ ]:
# First, activate the tardis-devel environment and run the first few cells to check for import issues

import astropy.units as u


In [25]:
from tardis.io.atom_data import download_atom_data

# Check if atomic data exists
print("Downloading atomic data...")
download_atom_data("kurucz_cd23_chianti_H_He_latest")

WARNING	Task(Task-3) tardis.io.atom_data.atom_web_download:atom_web_download.py:download_atom_data()- Atomic Data kurucz_cd23_chianti_H_He_latest already exists in /Users/wkerzend/projects/tardis/tardis-data/kurucz_cd23_chianti_H_He_latest.h5. Will not download - override with force_download=True.


In [26]:
from tardis.io.configuration.config_reader import Configuration

# Check if config file exists and can be read
config = Configuration.from_yaml("tardis_example.yml")
print("Configuration loaded successfully")

Configuration loaded successfully


In [27]:
# Enable rpacket tracking
config["montecarlo"]["tracking"]["track_rpacket"] = True
print("Rpacket tracking enabled")

Rpacket tracking enabled


In [28]:
from tardis import run_tardis

print("Running TARDIS simulation...")
sim = run_tardis(config, show_progress_bars=False, log_level="ERROR")
print("Simulation completed successfully")

Running TARDIS simulation...
Auto-detected VSCode environment


Auto-detected VSCode environment


BokehModel(combine_events=True, render_bundle={'docs_json': {'c005cffc-75b3-4829-bd93-2425ca951c1d': {'version…

TypingError: Failed in nopython mode pipeline (step: Handle with contexts)
[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mUnknown attribute 'line_before_id' of type instance.jitclass.RPacketTracker#32c9b4050<seed:int64,index:int64,status:array(int64, 1d, A),r:array(float64, 1d, A),nu:array(float64, 1d, A),mu:array(float64, 1d, A),energy:array(float64, 1d, A),shell_id:array(int64, 1d, A),interaction_type:array(int64, 1d, A),boundary_interaction:unaligned array(Record(event_id[type=int64;offset=0],current_shell_id[type=int64;offset=8],next_shell_id[type=int64;offset=16];24;False), 1d, A),num_interactions:int64,boundary_interactions_index:int64,event_id:int64,extend_factor:int64>
[1m
File "../../tardis/transport/montecarlo/packets/virtual_pac

In [36]:
# Test that the simulation works with the updated tracker
from tardis import run_tardis

print("Testing simulation with updated tracker...")
try:
    sim = run_tardis(config, show_progress_bars=False, log_level="ERROR")
    print("✓ Simulation completed successfully!")
except Exception as e:
    print(f"✗ Simulation failed: {e}")
    print(f"Error type: {type(e).__name__}")

Testing simulation with updated tracker...


BokehModel(combine_events=True, render_bundle={'docs_json': {'d3d0a698-189a-43eb-82e3-82c56d95691c': {'version…

✗ Simulation failed: Failed in nopython mode pipeline (step: Handle with contexts)
Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'initialize_tracker' of type instance.jitclass.RPacketLastInteractionTracker#32c9d4670<r:float64,nu:float64,energy:float64,shell_id:int64,interaction_type:int64,line_before_nu:float64,line_before_mu:float64,line_before_id:int64,line_after_nu:float64,line_after_mu:float64,line_after_id:int64,escat_before_mu:float64,escat_after_mu:float64,continuum_before_nu:float64,continuum_before_energy:float64,continuum_before_mu:float64,continuum_after_nu:float64,continuum_after_energy:float64,continuum_after_mu:float64,boundary_r:float64,boundary_nu:float64,boundary_energy:float64,boundary_mu:float64,boundary_from_shell_id:int64,boundary_to_shell_id:int64,line_interactions_count:int64,escat_interactions_count:int64,continuum_interactions_count:int64,boundary_interactions_count:int64>

In [ ]:
# Restart kernel and test again
import sys

# Clear imports
modules_to_reload = [name for name in sys.modules.keys() if name.startswith('tardis')]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]

# Now try to import and run again
from tardis.io.configuration.config_reader import Configuration

config = Configuration.from_yaml("tardis_example.yml")
config["montecarlo"]["tracking"]["track_rpacket"] = True

print("Testing simulation after cache clear...")
from tardis import run_tardis

try:
    sim = run_tardis(config, show_progress_bars=False, log_level="ERROR")
    print("✓ Simulation completed successfully!")
except Exception as e:
    print(f"✗ Simulation failed: {e}")
    print(f"Error type: {type(e).__name__}")
    # Show just the first few lines of the error for clarity
    error_str = str(e)
    print("First few lines of error:")
    for i, line in enumerate(error_str.split('\n')[:5]):
        print(f"  {line}")

In [ ]:
# Test the import fix
try:
    from tardis.transport.montecarlo.packets.trackers.r_packet_tracker import (
        generate_rpacket_tracker_list,
    )
    print("✓ Successfully imported r_packet_tracker!")

    # Try importing the main simulation module
    from tardis.simulation import Simulation
    print("✓ Successfully imported Simulation!")

except Exception as e:
    print(f"✗ Import failed: {e}")
    print(f"Error type: {type(e).__name__}")

✓ Successfully imported r_packet_tracker!


Initializing tabulator and plotly panel extensions for widgets to work


✓ Successfully imported Simulation!


In [ ]:
# Now test the full simulation with the fixes
from tardis import run_tardis
from tardis.io.configuration.config_reader import Configuration

config = Configuration.from_yaml("tardis_example.yml")
config["montecarlo"]["tracking"]["track_rpacket"] = True

print("Testing TARDIS simulation with fixed imports...")
try:
    sim = run_tardis(config, show_progress_bars=False, log_level="ERROR")
    print("✓ Simulation completed successfully!")

    # Quick check that we can access tracker data
    print(f"✓ Tracker data available with {len(sim.transport.transport_state.rpacket_tracker_df)} packets")

except Exception as e:
    print(f"✗ Simulation failed: {type(e).__name__}")
    # Show just the key part of the error
    error_lines = str(e).split('\n')
    if len(error_lines) > 5:
        print("Key error lines:")
        for line in error_lines[:5]:
            if line.strip():
                print(f"  {line}")
    else:
        print(f"Error: {e}")

Testing TARDIS simulation with fixed imports...
Auto-detected VSCode environment


Auto-detected VSCode environment


BokehModel(combine_events=True, render_bundle={'docs_json': {'599500b4-d084-4aa3-9757-a3b4ef418640': {'version…

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


✗ Simulation failed: TypingError
Key error lines:
  Failed in nopython mode pipeline (step: Handle with contexts)
  Failed in nopython mode pipeline (step: nopython frontend)
  Failed in nopython mode pipeline (step: nopython frontend)
  Unknown attribute 'track_boundary_crossing' of type instance.jitclass.RPacketTracker#33c211a90<seed:int64,index:int64,status:array(int64, 1d, A),r:array(float64, 1d, A),nu:array(float64, 1d, A),mu:array(float64, 1d, A),energy:array(float64, 1d, A),shell_id:array(int64, 1d, A),interaction_type:array(int64, 1d, A),boundary_interaction:unaligned array(Record(event_id[type=int64;offset=0],current_shell_id[type=int64;offset=8],next_shell_id[type=int64;offset=16];24;False), 1d, A),num_interactions:int64,boundary_interactions_index:int64,event_id:int64,extend_factor:int64>
  
